In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
import warnings
import math
from datetime import datetime

warnings.filterwarnings("ignore")


### Preparing meteo data


In [77]:
df_meteo = pd.read_csv('meteo.csv')
columns_to_drop = ['LC_TEMP_QCL0', 'LC_TEMP_QCL1', 'LC_TEMP_QCL2', 'LC_TEMP_QCL3',"LC_n","LC_DAILYRAIN","Minute","LC_RAD60"]
df_meteo = df_meteo.groupby(["Year","Month","Day","Hour"]).aggregate('mean').reset_index()
df_meteo = df_meteo.drop(df_meteo.index[-1])
df_meteo["LC_TEMP"] = df_meteo[['LC_TEMP_QCL0', 'LC_TEMP_QCL1', 'LC_TEMP_QCL2', 'LC_TEMP_QCL3']].mean(axis=1)
df_meteo = df_meteo.drop(columns=columns_to_drop)


In [78]:
df_meteo["hour_of_year"] = df_meteo.index

In [79]:
df_meteo

,Year,Month,Day,Hour,LC_HUMIDITY,LC_DWPTEMP,LC_RAD,LC_RAININ,LC_WINDDIR,LC_WINDSPEED,LC_TEMP,hour_of_year
0,2022,1,1,0,90.367865,11.501860,0.082452,0.000000,-17.124736,0.153679,13.037607,0
1,2022,1,1,1,89.504394,11.350650,0.093146,0.000007,-17.166960,0.229297,13.031624,1
2,2022,1,1,2,89.311072,11.132355,0.094903,0.000009,-14.453427,0.224025,12.838152,2
3,2022,1,1,3,89.400000,10.985421,0.085965,0.000000,-17.203509,0.183456,12.680371,3
4,2022,1,1,4,89.247803,10.636309,0.079086,0.000000,-10.896309,0.245185,12.348416,4
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2022,12,31,19,59.528716,8.152348,0.111486,0.000003,-32.202703,1.536892,16.028920,8755
8756,2022,12,31,20,58.145270,7.978159,0.113176,0.000000,-34.349662,1.521419,16.206959,8756
8757,2022,12,31,21,57.956303,7.815311,0.110924,0.000000,-33.694118,1.564739,16.083809,8757
8758,2022,12,31,22,56.662732,7.542850,0.118044,0.000005,-36.720067,1.549393,16.150651,8758


### Preparing events data


In [6]:
df_events = pd.read_csv('events.csv')


In [7]:
columns_to_drop = ["noise_event_laeq_model_id_unit","noise_event_laeq_primary_detected_class_unit","noise_event_laeq_primary_detected_certainty_unit","#object_id"]
df_events = df_events.drop(columns=columns_to_drop)
df_events = df_events.dropna(axis=0)
df_events = df_events[df_events["noise_event_laeq_primary_detected_certainty"] > 30]

In [9]:
df_events["description"].value_counts()

MP 01: Naamsestraat 35  Maxim       18340
MP 07: Naamsestraat 81              15868
MP 05: Calvariekapel KU Leuven       6956
MP 06: Parkstraat 2 La Filosovia     6791
MP 03: Naamsestraat 62 Taste         5387
MP 02: Naamsestraat 57 Xior          2848
MP08bis - Vrijthof                    198
Name: description, dtype: int64

In [4]:
def parse_date(date_string):
    return datetime.strptime(date_string, "%d/%m/%Y %H:%M:%S.%f")
start_date = pd.Timestamp(2022,1,1)

In [5]:
df_events['result_timestamp'] = df_events['result_timestamp'].apply(parse_date)
df_events['hour_of_year'] = (df_events['result_timestamp'] - start_date).dt.total_seconds() / 3600
df_events['hour_of_year']  = np.floor(df_events['hour_of_year'])
df_events_occur = df_events.drop(columns = ["noise_event_laeq_primary_detected_certainty","noise_event_laeq_model_id","result_timestamp"])
df_events_occur = pd.DataFrame(df_events_occur.value_counts()).reset_index().rename(columns = {0:"count"})
df_events_occur = df_events_occur.sort_values(["hour_of_year","description"])

In [286]:
df_events_occur_wide = df_events_occur.pivot(index=['hour_of_year',"description"], columns='noise_event_laeq_primary_detected_class', values='count').fillna(0)
df_events_occur_wide = df_events_occur_wide.reset_index()

### Preparing Noise data

In [92]:
df_noise_level = pd.read_csv("noise_level_q1.csv")

In [93]:
columns_to_drop = ["lamax_unit","laeq_unit","lceq_unit","lcpeak_unit","#object_id"]
df_noise_level = df_noise_level.drop(columns = columns_to_drop)

In [94]:

df_noise_level['result_timestamp'] = df_noise_level['result_timestamp'].apply(parse_date)
df_noise_level['hour_of_year'] = (df_noise_level['result_timestamp'] - start_date).dt.total_seconds() / 3600
df_noise_level['hour_of_year']  = np.floor(df_noise_level['hour_of_year'])



In [97]:
groups = df_noise_level.groupby('description')

dfs =[]
for name, group in groups:
    group_df = pd.DataFrame(group).groupby("hour_of_year").aggregate('mean')
    group_df["description"] = name
    dfs.append(group_df)
df = pd.concat(dfs)
df.to_csv("noise_level_q1.csv")

In [203]:
dfs = []
for i in range(1,5):
    file_name = "noise_level_q"+str(i)+".csv"
    df = pd.read_csv(file_name)
    dfs.append(df)

In [207]:
df_noise_level = pd.concat(dfs)
df_noise_level.to_csv("noise_level_ordered.csv")


In [89]:
df_noise_level = pd.read_csv("noise_level_ordered.csv",index_col = 0)
df_noise_level = pd.read_csv("df_noise_level.csv", index_col = 0)
df_noise_level = df_noise_level.sort_values(["hour_of_year","description"])
df_noise_level = df_noise_level.drop(columns="index")

### Preparing Final Dataset

In [85]:
final_df = pd.merge(df_noise_level, df_event_occ, on='hour_of_year', how='left')
final_df = final_df.fillna(0)
final_df = pd.merge(final_df, df_meteo, on='hour_of_year', how='left')


In [250]:
final_df

,hour_of_year,lamax,laeq,lceq,lcpeak,description,Year,Month,Day,Hour,LC_HUMIDITY,LC_DWPTEMP,LC_RAD,LC_RAININ,LC_WINDDIR,LC_WINDSPEED,LC_TEMP
0,0.0,59.975000,56.855417,62.860083,76.341250,MP 03: Naamsestraat 62 Taste,2022,1,1,0,90.367865,11.501860,0.082452,0.000000,-17.124736,0.153679,13.037607
1,0.0,53.960417,50.482500,56.161250,70.987375,MP 05: Calvariekapel KU Leuven,2022,1,1,0,90.367865,11.501860,0.082452,0.000000,-17.124736,0.153679,13.037607
2,0.0,53.812083,50.967083,55.806042,69.958500,MP 06: Parkstraat 2 La Filosovia,2022,1,1,0,90.367865,11.501860,0.082452,0.000000,-17.124736,0.153679,13.037607
3,0.0,50.140833,47.502083,55.115875,67.835667,MP 07: Naamsestraat 81,2022,1,1,0,90.367865,11.501860,0.082452,0.000000,-17.124736,0.153679,13.037607
4,1.0,53.062500,50.729583,58.584667,71.182042,MP 03: Naamsestraat 62 Taste,2022,1,1,1,89.504394,11.350650,0.093146,0.000007,-17.166960,0.229297,13.031624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54720,8759.0,61.701250,59.173333,67.984417,80.226958,MP 04: His & Hears,2022,12,31,23,53.859083,7.059032,0.122241,0.000000,-34.101868,1.500136,16.428192
54721,8759.0,59.319167,56.234167,63.158833,76.554167,MP 05: Calvariekapel KU Leuven,2022,12,31,23,53.859083,7.059032,0.122241,0.000000,-34.101868,1.500136,16.428192
54722,8759.0,57.570417,55.180000,61.123083,74.051250,MP 06: Parkstraat 2 La Filosovia,2022,12,31,23,53.859083,7.059032,0.122241,0.000000,-34.101868,1.500136,16.428192
54723,8759.0,55.437083,53.020833,61.964167,74.617250,MP 07: Naamsestraat 81,2022,12,31,23,53.859083,7.059032,0.122241,0.000000,-34.101868,1.500136,16.428192


In [298]:
final_df = pd.read_csv("final_df.csv",index_col=0)
final_df = final_df.drop(columns = ["number_of_events"])

In [300]:
final_df = pd.merge(final_df, df_events_occur_wide, on=['hour_of_year','description'], how='left').fillna(0)


In [335]:
final_df.to_csv("final_df.csv")